In [1]:
import django
import os
import json
os.environ['DJANGO_SETTINGS_MODULE'] = 'django_settings'
django.setup()

In [2]:
import pandas as pd
import requests
from io import BytesIO
from boto3.session import Session
import boto3
import botocore

In [3]:
from data_management.models import *
from loader import load_credential

In [4]:
ACCESS_KEY = load_credential("AWS_ACCESS_KEY_ID","")
SECRET_ACCESS_KEY = load_credential("AWS_SECRET_ACCESS_KEY","")

In [5]:
CroppedImage.objects.select_related('origin_source')

<QuerySet [<CroppedImage: CroppedImage object (178)>, <CroppedImage: CroppedImage object (179)>, <CroppedImage: CroppedImage object (180)>, <CroppedImage: CroppedImage object (181)>, <CroppedImage: CroppedImage object (182)>, <CroppedImage: CroppedImage object (183)>, <CroppedImage: CroppedImage object (184)>, <CroppedImage: CroppedImage object (185)>, <CroppedImage: CroppedImage object (186)>, <CroppedImage: CroppedImage object (187)>, <CroppedImage: CroppedImage object (188)>, <CroppedImage: CroppedImage object (189)>, <CroppedImage: CroppedImage object (190)>, <CroppedImage: CroppedImage object (191)>, <CroppedImage: CroppedImage object (192)>, <CroppedImage: CroppedImage object (193)>, <CroppedImage: CroppedImage object (194)>, <CroppedImage: CroppedImage object (195)>, <CroppedImage: CroppedImage object (196)>, <CroppedImage: CroppedImage object (197)>, '...(remaining elements truncated)...']>

In [6]:
ProductCategories.objects.select_related('cropped_source')

<QuerySet [<ProductCategories: ProductCategories object (11)>, <ProductCategories: ProductCategories object (12)>, <ProductCategories: ProductCategories object (13)>, <ProductCategories: ProductCategories object (14)>, <ProductCategories: ProductCategories object (15)>, <ProductCategories: ProductCategories object (16)>, <ProductCategories: ProductCategories object (17)>, <ProductCategories: ProductCategories object (18)>, <ProductCategories: ProductCategories object (19)>, <ProductCategories: ProductCategories object (20)>, <ProductCategories: ProductCategories object (21)>, <ProductCategories: ProductCategories object (22)>, <ProductCategories: ProductCategories object (23)>, <ProductCategories: ProductCategories object (24)>, <ProductCategories: ProductCategories object (25)>, <ProductCategories: ProductCategories object (26)>, <ProductCategories: ProductCategories object (27)>, <ProductCategories: ProductCategories object (28)>, <ProductCategories: ProductCategories object (29)>, <

In [7]:
class DataExtractSave():
    def __init__(self):
        self.left = []
        self.top = []
        self.right = []
        self.bottom = []
        self.origin_url = []
        self.cropped_url = []
        self.shape_source = []
        self.cover_source = []
        self.charm_source = []
        self.pattern_source = []
        self.width = []
        self.height = []
    
    def get_image_size(self, resp):
        from PIL import Image
        byteImgIO = BytesIO()
        byteImg = Image.open(BytesIO(resp.content))
        byteImg.convert('RGB').save(byteImgIO, "JPEG")
        byteImgIO.seek(0)
        byteImg = byteImgIO.read()
        dataBytesIO = BytesIO(byteImg)
        image = Image.open(dataBytesIO)
        image = image.convert('RGB')
        width, height = image.size
        return width, height
    
    def get_boxinfo_and_append(self, obj, url):
        self.origin_url.append(url)
        resp = requests.get(url)
        w, h = self.get_image_size(resp)
        self.width.append(w)
        self.height.append(h)
        self.left.append(obj.left*w)
        self.top.append(obj.top*h)
        self.right.append(obj.right*w)
        self.bottom.append(obj.bottom*h)
        
    def get_category_and_append(self, obj, url):
        self.cropped_url.append(url)
        self.shape_source.append(obj.shape_source_id)
    
    def save_boxinfo_to_csv(self, filename):
        dic = {}
        dic['left'] = self.left
        dic['top'] = self.top
        dic['right'] = self.right
        dic['bottom'] = self.bottom
        dic['origin_url'] = self.origin_url
        dic['width'] = self.width
        dic['height'] = self.height
    
        test_data_pd=pd.DataFrame.from_dict(dic)
        print(filename)
        test_data_pd.to_csv(filename, mode='w', index=False)
        print('...completed')
        print('\n')
    
    def save_category_to_csv(self, filename):
        dic = {}
        dic['cropped_url'] = self.cropped_url
        dic['shape_source'] = self.shape_source
        
        test_data_pd=pd.DataFrame.from_dict(dic)
        print(filename)
        test_data_pd.to_csv(filename, mode='w', index=False)
        print('...completed')
        print('\n')
        
    def save_boxing_data_to_csv(self, filename, queryset):       
        print(queryset.count(), ' 개 data extract start')
        i = 0
        for c in queryset:
            i +=1
            if i%50==0:
                print('making... ',i,'/ ',c)
            if c.origin_source.image:
                url = c.origin_source.image.url
                self.get_boxinfo_and_append(c, url)
            elif c.origin_source.s3_image_url:
                url = c.origin_source.s3_image_url
                self.get_boxinfo_and_append(c, url)
        self.save_boxinfo_to_csv(filename)
        
    def save_labeling_data_to_csv(self, filename, queryset):       
        print(queryset.count(), ' 개 data extract start')
        i = 0
        for c in queryset:
            i +=1
            if i%50==0:
                print('making... ',i,'/ ',c)
            if c.cropped_source.image:
                url = c.cropped_source.image.url
                self.get_category_and_append(c, url)
        self.save_category_to_csv(filename)

In [8]:
class S3DownloadManager(): # download from queryset
        def __init__(self):
            self.access_key = load_credential("AWS_ACCESS_KEY_ID","")
            self.secret_access_key = load_credential("AWS_SECRET_ACCESS_KEY","")
            self.s3 =  boto3.resource('s3',
                                    aws_access_key_id=ACCESS_KEY,
                                    aws_secret_access_key=SECRET_ACCESS_KEY)
            self.temp_bucket_list = self.get_s3_file_temp_bucket()
            self.mondedata_bucket_list = self.get_s3_file_mondedata_bucket()
            self.cropped_bucket_list = self.get_s3_file_cropdata_bucket()
            
        def get_s3_file_temp_bucket(self):
            lists = []
            for s3_file in self.s3.Bucket('temp-originalimage').objects.all():
                lists.append(s3_file.key)
            return lists
        
        def get_s3_file_mondedata_bucket(self):
            lists = []
            for s3_file in self.s3.Bucket('monde-data').objects.all():
                if s3_file.key.split('/')[0] == 'original-bag-images-dev':
                    lists.append(s3_file.key)
            return lists
        
        def get_s3_file_cropdata_bucket(self):
            lists = []
            for s3_file in self.s3.Bucket('monde-data').objects.all():
                if s3_file.key.split('/')[0] == 'cropped-bag-images-dev':
                    lists.append(s3_file.key)
            return lists
        
        # s3_image_url 이 존재할 때
        def get_origin_list_from_queryset(self, queryset, n):
            new_file_list_t = []
            new_file_list_m = []
            
            qs_temp_bucket = queryset.filter(origin_source__s3_image_url__isnull=False)
            temp_bucket_list = []
            qs_mondedata_bucket = queryset.exclude(origin_source__image="")
            mondedata_bucket_list = []
            
            for q in qs_temp_bucket:
                q = q.origin_source
                name = q.s3_image_url.split('/')[-1]
                temp_bucket_list.append(name)
                
            for q in qs_mondedata_bucket:
                q = q.origin_source
                name = q.image.name
                mondedata_bucket_list.append(name)
            
            pre_temp_bucket_list = self.temp_bucket_list
            
            #temp_bucket
            for i in range(len(pre_temp_bucket_list)):
                for j in range(len(temp_bucket_list)):
                    if(pre_temp_bucket_list[i] == temp_bucket_list[j]):
                        new_file_list_t.append(pre_temp_bucket_list[i])
        
            pre_mondedata_bucket_list = self.mondedata_bucket_list
            
            #mondedata_bucket
            for i in range(len(pre_mondedata_bucket_list)):
                for j in range(len(mondedata_bucket_list)):
                    if(pre_mondedata_bucket_list[i] == mondedata_bucket_list[j]):
                        new_file_list_m.append(pre_mondedata_bucket_list[i])
                        
            if n == 1: # temp bucket
                return new_file_list_t
            elif n == 2:
                return new_file_list_m
            else:
                print('ERROR')
                
        def get_cropped_list_from_queryset(self, queryset):
            new_file_list = []
            
            qs_cropped_bucket = queryset.exclude(cropped_source__image="")
            cropped_bucket_list_m = []
            
            for q in qs_cropped_bucket:
                q = q.cropped_source
                name = q.image.name
                cropped_bucket_list_m.append(name)
            
            pre_cropped_bucket_list = self.cropped_bucket_list
            
            for i in range(len(pre_cropped_bucket_list)):
                for j in range(len(cropped_bucket_list_m)):
                    if(pre_cropped_bucket_list[i] == cropped_bucket_list_m[j]):
                        new_file_list.append(pre_cropped_bucket_list[i])
                        
            return new_file_list
    
        def download_origin_s3_files_from_queryset(self, local_path, queryset):
            # local_path 는 jupyter를 실행한 폴더 기준
            # 폴더 경로는 \\로 끝나야 함 (for Window)
            file_list_t = self.get_origin_list_from_queryset(queryset, 1)
            file_list_m = self.get_origin_list_from_queryset(queryset, 2)
            print('Start download images')
            i = 0
            for file in file_list_t:
                i += 1
                KEY = file
                dest_pathname = os.path.join(local_path, KEY)
                if i % 100 == 0:
                    print(i)
                    print(KEY)
                try:
                    self.s3.Bucket('temp-originalimage').download_file(KEY, dest_pathname)
                except botocore.exceptions.ClientError as e:
                    if e.response['Error']['Code'] == "404":
                        print("The object does not exist.")
                    else:
                        print('----')
                        raise
            print(i,' 개 downloaded from temp bucket')
        
            i = 0
            for file in file_list_m:
                i += 1
                KEY = file
                path_key = KEY.split('/')[1]
                dest_pathname = os.path.join(local_path, path_key)
                if i % 100 == 0:
                    print(i)
                    print(KEY)
                try:
                    self.s3.Bucket('monde-data').download_file(KEY, dest_pathname)
                except botocore.exceptions.ClientError as e:
                    if e.response['Error']['Code'] == "404":
                        print("The object does not exist.")
                    else:
                        print('----')
                        raise
            print(i,' 개 downloaded from monde-data bucket')
            print('DONE')
            
        def download_cropped_s3_files_from_queryset(self, local_path, queryset):
            # local_path 는 jupyter를 실행한 폴더 기준
            # 폴더 경로는 \\로 끝나야 함 (for Window)
            file_list = self.get_cropped_list_from_queryset(queryset)
            
            print('Start download images')
            i = 0
            for file in file_list:
                i += 1
                KEY = file
                dest_pathname = os.path.join(local_path, KEY)
                if i % 100 == 0:
                    print(i)
                    print(KEY)
                try:
                    self.s3.Bucket('monde-data').download_file(KEY, dest_pathname)
                except botocore.exceptions.ClientError as e:
                    if e.response['Error']['Code'] == "404":
                        print("The object does not exist.")
                    else:
                        print('----')
                        raise
            print(i,' 개 downloaded from monde-data bucket')
            print('DONE')

## Basic extract

In [11]:
# instance생성 및 queryset 선언
extractmanager1= DataExtractSave()
imagedownloadmanager1 = S3DownloadManager()
queryset1 = CroppedImage.objects.filter(origin_source__image_url__isnull=False)

In [ ]:
# queryset info download csv form
extractmanager1.save_data_to_csv("crop_download_test.csv", queryset1)

In [ ]:
# image download to 'data' folder => 다운받기 전 'data'라는 폴더 생성해야 합니다.
imagedownloadmanager1.download_s3_files_from_queryset('./data/', queryset1)

## image review (몸체만 정확히 박싱된 데이터) extract

In [29]:
extractmanager2= DataExtractSave()
imagedownloadmanager2 = S3DownloadManager()
queryset2 = CroppedImage.objects.filter(origin_source__image_review=True)
a = ShapeTag.objects.first()
q = CroppedImage.objects.filter(categories__shape_source=a)

In [30]:
extractmanager2.save_data_to_csv("test.csv", q)
imagedownloadmanager2.download_s3_files_from_queryset('./data/', q)

223  개 data download start
making...  50 /  CroppedImage object (10088)
making...  100 /  CroppedImage object (10141)
making...  150 /  CroppedImage object (10194)
making...  200 /  CroppedImage object (10248)
test.csv
downloaded
Start download images
100
square_216_5Y5I61OPPTU_1.jpg
186  개 downloaded from temp bucket
37  개 downloaded from monde-data bucket
DONE


## labeling data extract

In [11]:
extractmanager3 = DataExtractSave()
imagedownloadmanager3 = S3DownloadManager()
queryset3 = ProductCategories.objects.filter(shape_source_id__isnull=False)
extractmanager3.save_labeling_data_to_csv("shape_labeling.csv", queryset3)
imagedownloadmanager3.download_cropped_s3_files_from_queryset('./labeling_data/', queryset3)

1523  개 data extract start
making...  50 /  ProductCategories object (60)
making...  100 /  ProductCategories object (112)
making...  150 /  ProductCategories object (162)
making...  200 /  ProductCategories object (212)
making...  250 /  ProductCategories object (262)
making...  300 /  ProductCategories object (312)
making...  350 /  ProductCategories object (362)
making...  400 /  ProductCategories object (412)
making...  450 /  ProductCategories object (462)
making...  500 /  ProductCategories object (512)
making...  550 /  ProductCategories object (562)
making...  600 /  ProductCategories object (612)
making...  650 /  ProductCategories object (662)
making...  700 /  ProductCategories object (712)
making...  750 /  ProductCategories object (762)
making...  800 /  ProductCategories object (812)
making...  850 /  ProductCategories object (862)
making...  900 /  ProductCategories object (912)
making...  950 /  ProductCategories object (962)
making...  1000 /  ProductCategories object 

In [ ]:
# TODO : 현재는 박싱 서버에서 쌓은 데이터 기반으로 사용 가능하지만, 추후 mondebro에서 태깅한 데이터도 사용 가능하게 수정해야 합니다.